# Generics

In [1]:
!go version

go version go1.22.2 linux/amd64


# `any`

In [2]:
import "strings"
import "fmt"

// example in 'Learning Go'

type Orderable interface {
	Order(any) int // any
}

type Tree struct {
	val Orderable
	left, right *Tree
}

func (t *Tree) Insert(val Orderable) *Tree {
	if t == nil {
		return &Tree{val: val}
	}

	switch comp := val.Order(t.val); {
	case comp < 0:
		t.left = t.left.Insert(val)
	case comp > 0:
		t.right = t.right.Insert(val)
	}

	return t
}

// int
type OrderableInt int

func (oi OrderableInt) Order(val any) int {
	return int(oi - val.(OrderableInt))
}

// string

type OrderableString string

func (os OrderableString) Order(val any) int {
	return strings.Compare(string(os), val.(string))
}

%%
var it *Tree
it = it.Insert(OrderableInt(5))
fmt.Printf("%v\n", it)
it = it.Insert(OrderableString("nope"))
fmt.Printf("%v\n", it)

&{5 <nil> <nil>}


panic: interface conversion: interface {} is main.OrderableInt, not string

goroutine 1 [running]:
main.OrderableString.Order(...)
	 [[ Cell [2] Line 42 ]] /tmp/gonb_f7089ad0/main.go:24
main.(*Tree).Insert(0xc000130000, {0x4cb498, 0x4caec0})
	 [[ Cell [2] Line 20 ]] /tmp/gonb_f7089ad0/main.go:32 +0x4d
main.main()
	 [[ Cell [2] Line 49 ]] /tmp/gonb_f7089ad0/main.go:48 +0xd1
exit status 2


# `Stack[T any]`

In [3]:
// example in 'Learning Go'

// type paramter: T
// type constraint: any
type Stack[T any] struct {
	vals []T
}

func (s *Stack[T]) Push(val T) {
	s.vals = append(s.vals, val)
}

func (s *Stack[T]) Pop() (T, bool) {
	if len(s.vals) == 0 {
		var zero T
		return zero, false
	}
	top := s.vals[len(s.vals) - 1]
	s.vals = s.vals[:len(s.vals) - 1]
	return top, true
}

%%
var intStack Stack[int]
intStack.Push(10)
intStack.Push(20)
intStack.Push(30)
v, ok := intStack.Pop()
fmt.Println(v, ok)
fmt.Printf("%v\n", intStack)

30 true
{[10 20]}


In [4]:
%%
var intStack Stack[int]
intStack.Push("nope")

# gonb_f7089ad0 

 

 Cell[4]: Line 3 
 ./main.go:62:15: cannot use "nope" (untyped string constant) as int value in argument to intStack.Push
 
func main() {
	flag.Parse()
var intStack Stack[int]
 intStack.Push("nope")
 
}

ERROR: failed to run "/home/zhoujiagen/go/bin/go build -o /tmp/gonb_f7089ad0/gonb_f7089ad0": exit status 1

In [5]:
func (s Stack[T]) Contains(val T) bool {
	for _,v := range(s.vals) {
		if v == val {
			return true
		}
	}
	return false
}

# gonb_f7089ad0 

 

 Cell[5]: Line 3 
 ./main.go:60:6: invalid operation: v == val (incomparable types in type set)
 

func (s Stack[T]) Contains(val T) bool {
	for _,v := range(s.vals) {
 if v == val {
 return true
 }

ERROR: failed to run "/home/zhoujiagen/go/bin/go build -o /tmp/gonb_f7089ad0/gonb_f7089ad0": exit status 1

In [6]:
import "fmt"

type Stack2[T comparable] struct { // comparable interface
	vals []T
}

func (s *Stack2[T]) Push(val T) {
	s.vals = append(s.vals, val)
}

func (s *Stack2[T]) Pop() (T, bool) {
	if len(s.vals) == 0 {
		var zero T
		return zero, false
	}
	top := s.vals[len(s.vals) - 1]
	s.vals = s.vals[:len(s.vals) - 1]
	return top, true
}

func (s Stack2[T]) Contains(val T) bool {
	for _,v := range(s.vals) {
		if v == val {
			return true
		}
	}
	return false
}

%%
var s Stack2[int]
s.Push(10)
s.Push(20)
s.Push(30)
fmt.Println(s.Contains(10))
fmt.Println(s.Contains(5))

true
false


# generic functions

In [7]:
// example in 'Learning Go'
import "fmt"

func Map[T1, T2 any](s []T1, f func(T1) T2) []T2 {
	r := make([]T2, len(s))
	for i,v := range s {
		r[i] = f(v)
	}
	return r
}

func Reduce[T1, T2 any](s []T1, initializer T2, f func(T2, T1) T2) T2 {
	r := initializer
	for _, v := range s {
		r = f(r, v)
	}
	return r
}

func Filter[T any](s []T, f func(T) bool) []T {
	var r []T
	for _, v := range s {
		if f(v) {
			r = append(r, v)
		}
	}
	return r
}

%% 
words := []string{"One", "Potato", "Two", "Potato"}
filtered := Filter(words, func(s string) bool {
	return s != "Potato"
})
fmt.Println(filtered)
lengths := Map(filtered, func(s string) int {
	return len(s)
})
fmt.Println(lengths)
sum := Reduce(lengths, 0, func(acc int, val int) int {
	return acc + val
})
fmt.Println(sum)

[One Two]
[3 3]
6


# generics and interfaces

In [8]:
// example in 'Learning Go'
import (
	"fmt"
	"math"
)

type Pair[T fmt.Stringer] struct {
	Val1 T
	Val2 T
}

// interface with type parameters
type Differ[T any] interface {
	fmt.Stringer
	Diff(T) float64
}

func FindCloser[T Differ[T]](pair1, pair2 Pair[T]) Pair[T] {
	d1 := pair1.Val1.Diff(pair1.Val2)
	d2 := pair2.Val1.Diff(pair2.Val2)
	if d1 < d2 {
		return pair1
	}
	return pair2
}

// implementation

type Point2D struct {
	X, Y int
}

func (p2 Point2D) String() string {
	return fmt.Sprintf("{%d,%d}", p2.X, p2.Y)
}

func (p2 Point2D) Diff(from Point2D) float64 {
	x := p2.X - from.X
	y := p2.Y - from.Y
	return math.Sqrt(float64(x*x) + float64(y*y))
}

type Point3D struct {
	X,Y,Z int
}

func (p3 Point3D) String() string {
	return fmt.Sprintf("{%d,%d,%d}", p3.X, p3.Y, p3.Z)
}

func (p3 Point3D) Diff(from Point3D) float64 {
	x := p3.X - from.X
	y := p3.Y - from.Y
	z := p3.Z - from.Z
	return math.Sqrt(float64(x*x) + float64(y*y) + float64(z*z))
}

%%
pair2Da := Pair[Point2D]{Point2D{1,1}, Point2D{5,5}}
pair2Db := Pair[Point2D]{Point2D{10,10}, Point2D{15,5}}
closer := FindCloser(pair2Da, pair2Db)
fmt.Println(closer)

pair3Da := Pair[Point3D]{Point3D{1,1,10}, Point3D{5,5,0}}
pair3Db := Pair[Point3D]{Point3D{10,10,10}, Point3D{11,5,0}}
closer2 := FindCloser(pair3Da, pair3Db)
fmt.Println(closer2)

{{1,1} {5,5}}
{{10,10,10} {11,5,0}}


# type terms

In [9]:
import (
	"errors"
	"fmt"
)

// can only used as type constraints
type Integer interface {
	// type element: compose of type terms
	// exact match
	// int | int8 | int16 | int32 | int64 |
	// uint | uint8 | uint16 | uint32 | uint64 | uintptr
	
	// consider underlying type
	~int | ~int8 | ~int16 | ~int32 | ~int64 |
	~uint | ~uint8 | ~uint16 | ~uint32 | ~uint64 | ~uintptr
}

func divAndReainder[T Integer](num, denom T) (T, T, error) {
	if denom == 0 {
		return 0,0,errors.New("cannot divide by zero")
	}
	// operators: /, %
	return num / denom, num % denom, nil
}


type MyInt int

%%
var a uint = 18_446_744_073_709_551_615
var b uint = 9_223_372_036_854_775_808
fmt.Println(divAndReainder(a, b))

var myA MyInt = 10
var myB MyInt = 20
fmt.Println(divAndReainder(myA, myB))

1 9223372036854775807 <nil>
0 10 <nil>


In [10]:
import "fmt"

// both type elements and method elements in interface used for type parameter
type PrintableInt interface {
	~int
	String() string
}

// declare a type parameter interface that cannot instantiate
type ImpossiblePrintableInt interface {
	int
	String() string
}

type ImpossibleStruct[T ImpossiblePrintableInt] struct {
	val T
}

type MyInt int
func (mi MyInt) String() string {
	return fmt.Sprint(mi)
}

%%
s := ImpossibleStruct[int]{10}
s2 := ImpossibleStruct[MyInt]{10}
fmt.Printf("%v %v\n", s, s2)

# gonb_f7089ad0 

 

 Cell[10]: Line 25 
 ./main.go:194:23: int does not satisfy ImpossiblePrintableInt (missing method String)
 

func main() {
	flag.Parse()
 s := ImpossibleStruct[int]{10}
 s2 := ImpossibleStruct[MyInt]{10}
fmt.Printf("%v %v\n", s, s2)

 

 

 Cell[10]: Line 26 
 ./main.go:195:24: MyInt does not satisfy ImpossiblePrintableInt (possibly missing ~ for int in ImpossiblePrintableInt)
 
func main() {
	flag.Parse()
s := ImpossibleStruct[int]{10}
 s2 := ImpossibleStruct[MyInt]{10}
 fmt.Printf("%v %v\n", s, s2)

ERROR: failed to run "/home/zhoujiagen/go/bin/go build -o /tmp/gonb_f7089ad0/gonb_f7089ad0": exit status 1

In [ ]:
// type terms can also be:
// slices, maps, arrays, channels, structs, functions

# type inference

In [11]:
// example in 'Learning Go'
import "fmt"

type Integer interface{
	int | int8 | int16 | int32 | int64 |
	uint | uint8 | uint16 | uint32 | uint64 
}

func Convert[T1, T2 Integer](in T1) T2 {
	return T2(in)
}

%%
var a int = 10
b := Convert[int, int64](a) // cannot infer return type
fmt.Println(b)

10


# limit constants

In [12]:
// example in 'Learning Go'
type Integer interface{
	int | int8 | int16 | int32 | int64 |
	uint | uint8 | uint16 | uint32 | uint64 
}

// cannot convert 1_000 (untyped int constant 1000) to type T
// func Plus1000[T Integer](in T) T {
// 	return in + 1_000
// }

func Plus100[T Integer](in T) T {
	return in + 100
}

# generic functions with generic data structures

In [13]:
%list

Memorized Definitions

Imports 
 
 errors 
 fmt 
 math 
 strings

Types 
 
 Differ 
 Integer 
 MyInt 
 Orderable 
 OrderableInt 
 OrderableString 
 Pair 
 Point2D 
 Point3D 
 Stack 
 Stack2 
 Tree

Functions 
 
 Convert 
 Filter 
 FindCloser 
 Map 
 OrderableInt~Order 
 OrderableString~Order 
 Plus100 
 Point2D~Diff 
 Point2D~String 
 Point3D~Diff 
 Point3D~String 
 Reduce 
 Stack2[T]~Pop 
 Stack2[T]~Push 
 Stack[T]~Pop 
 Stack[T]~Push 
 Tree~Insert 
 divAndReainder 
 unknown~Contains

In [14]:
// example in 'Learing Go'

import (
	"fmt"
	"cmp"
)

// generic function
type OrderableFunc[T any] func(t1, t2 T) int

// generic struct
// WARN: rename Node to TNode, Tree to TTree due to previous definitions
type TNode[T any] struct {
	val T
	left, right *TNode[T]
}

type TTree[T any] struct {
	f OrderableFunc[T]
	root *TNode[T]
}

// consturct a new tree
func NewTree[T any](f OrderableFunc[T]) *TTree[T] {
	return &TTree[T]{
		f: f,
	}
}

func (t *TTree[T]) Add(v T) {
	t.root = t.root.Add(t.f, v)
}

func (t *TTree[T]) Contains(v T) bool {
	return t.root.Contains(t.f, v)
}

func (n *TNode[T]) Add(f OrderableFunc[T], v T) *TNode[T] {
	if n == nil {
		return &TNode[T]{val: v}
	}

	switch r := f(v, n.val); {
	case r <= -1:  // new value in less than current value
		n.left = n.left.Add(f, v)
	case r >= 1:
		n.right = n.right.Add(f, v)
	}
	return n
}

func (n *TNode[T]) Contains(f OrderableFunc[T], v T) bool {
	if n == nil {
		return false
	}

	switch r := f(v, n.val); {
	case r <= -1:
		return n.left.Contains(f, v)
	case r >= 1:
		return n.right.Contains(f, v)
	}
	return true
}

%%
// int
t1 := NewTree(cmp.Compare[int])
fmt.Printf("%v\n", t1)
t1.Add(10)
t1.Add(30)
t1.Add(15)

fmt.Println(t1.Contains(15))
fmt.Println(t1.Contains(40))


// struct

&{0x48aee0 <nil>}
true
false


In [ ]:
// !go run test_topics.go

In [ ]:
%%
t1 := NewTree(cmp.Compare[int])
fmt.Printf("%v\n", t1)
t1.Add(10)
t1.Add(30)
t1.Add(15)

fmt.Println(t1.Contains(15))
fmt.Println(t1.Contains(40))

# `comparable`
* more info: [All your comparable types](https://go.dev/blog/comparable)

In [16]:
import "fmt"

type Thinger interface {
	Thing()
}

type ThingerInt int

func (t ThingerInt) Thing() {
	fmt.Println("ThingInt:", t)
}

type ThingerSlice []int

func (t ThingerSlice) Thing() {
	fmt.Println("ThingSlice:", t)
}


func Comparer[T comparable](t1, t2 T) {
	if t1 == t2 {
		fmt.Println("equal!")
	}
}

In [17]:
%%
var a int = 10
var b int = 10
Comparer(a, b)

var a2 ThingerInt = 20
var b2 ThingerInt = 20
Comparer(a2, b2)

equal!
equal!


In [18]:
%%
var a3 ThingerSlice = []int{1,2,3}
var b3 ThingerSlice = []int{1,2,3}
Comparer(a3, b3)

# gonb_f7089ad0 

 

 Cell[18]: Line 4 
 ./main.go:253:9: ThingerSlice does not satisfy comparable
 
	flag.Parse()
var a3 ThingerSlice = []int{1,2,3}
var b3 ThingerSlice = []int{1,2,3}
 Comparer(a3, b3)
 
}

ERROR: failed to run "/home/zhoujiagen/go/bin/go build -o /tmp/gonb_f7089ad0/gonb_f7089ad0": exit status 1

In [ ]:
%%
var a2 ThingerInt = 20
var b2 ThingerInt = 20
var a4 Thinger = a2 // assign ThingerInt to Thinger
var b4 Thinger = b2
Comparer(a4, b4)

equal!


In [21]:
%%
var a3 ThingerSlice = []int{1,2,3}
var b3 ThingerSlice = []int{1,2,3}
var a4 Thinger = a3 // assign ThingerSlice to Thinger
var b4 Thinger = b3
Comparer(a4, b4)

panic: runtime error: comparing uncomparable type main.ThingerSlice

goroutine 1 [running]:
main.Comparer[...]({0x4ca1a8?, 0xc00011cf28?}, {0x4ca1a8?, 0xc00011cf10?})
	 [[ Cell [16] Line 21 ]] /tmp/gonb_f7089ad0/main.go:61 +0x35
main.main()
	 [[ Cell [21] Line 6 ]] /tmp/gonb_f7089ad0/main.go:255 +0x106
exit status 2


# NOT INCLUDED FEATURES
* operator overloading
* additional type parameters on methods
* variadic type parameters
* specialization: function/method be overloaded with one or more type-specific version in addition to the generic version
* currying: paritially instantiate a function or type by specifying some of the type parameters
* metaprogramming: specify code that runs at compile time to produce code that runs at runtime

# Performance Impact

In [23]:
type Ager interface {
	age() int
}

func doubleAge(a Ager) int {
	return a.age() * 2
}

func doubleAgeGeneric[T Ager](a T) int {
	return a.age() * 2
}

type SAger struct {
	agei int
}

func (sa SAger) age() int {
	return sa.agei
}

In [27]:
import "testing"

var sa = SAger{agei: 42}

func BenchmarkDoubleAg(b *testing.B) {
	doubleAge(sa)
}

func BenchmarkDoubleAgGeneric(b *testing.B) {
	doubleAgeGeneric(sa)
}


%test

goos: linux
goarch: amd64
pkg: gonb_f7089ad0
cpu: 13th Gen Intel(R) Core(TM) i7-13700H
BenchmarkDoubleAg
BenchmarkDoubleAg-20           	1000000000	         0.0000002 ns/op
BenchmarkDoubleAgGeneric
BenchmarkDoubleAgGeneric-20    	1000000000	         0.0000002 ns/op
PASS


# API Impact
* `any`, `comparable`
* packages
  * `slices`
  * `maps`
  * `sync`
  * ...